# Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# When to use a Sequential model
- A `Sequential` model is appropriate for **a plain stack of layers** where each layer has **exactly one input tensor and one output tensor**.
- A `Sequential` model is **not appropriate** when:
    - The model has multiple inputs or outputs
    - Any of the layers has multiple inputs or outputs
    - You need to do layer sharing
    - You want non-linear topology (e.g. a residual connection, a multi-brach model, etc.)

# Creating a Sequential model
- You can create a Sequential model by passing a list of layers to the `Sequential` constructor.

In [2]:
# Define a Sequential model with 3 layers
model = keras.models.Sequential([
    layers.Dense(2, activation='relu', name='layer1'),
    layers.Dense(3, activation='relu', name='layer2'),
    layers.Dense(4, name='layer3')
])

# Call model on a test input
X = tf.ones((3,3))
y = model(X)

- You can also create a Sequential model incrementally vua the `add()` method.

In [3]:
model = keras.models.Sequential()
model.add(layers.Dense(2, activation='relu', name='layer1'))
model.add(layers.Dense(3, activation='relu', name='layer2'))
model.add(layers.Dense(4, name='layer3'))

- Note that there is also a corresponding `pop()` method to remove layers.
    - A Sequential model behaves much like a list of layers.

In [4]:
model.pop()
len(model.layers)

2

# Specifying the input shape in advance
- Generally, all layers in Keras need to know the shape of their inputs in order to be able to create their weights. 
- So when you create a layer like the following example, initially, it has no weights.

In [5]:
layer = layers.Dense(3)
layer.weights

[]

- It creates its weights the first time it is called on an input, since the shape of the weights depends on the shape of the inputs.

In [6]:
X = tf.ones((1, 4))
y = layer(X)

layer.weights

[<tf.Variable 'dense/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.00261897,  0.5110556 , -0.46288446],
        [ 0.73821807, -0.19643867, -0.41821212],
        [-0.30663127,  0.6883552 ,  0.71600723],
        [-0.7568406 , -0.63529146, -0.23348731]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

- Naturally, this also applies to Sequential models. 
- When you instantiate a Sequential model without an input shape, it isn't "built". 
    - It has no weights (and calling `model.weights` results in an error stating just this). 
- The weights are created when the model first sees some input data.

In [7]:
model = keras.models.Sequential([
    layers.Dense(2, activation='relu', name='layer1'),
    layers.Dense(3, activation='relu', name='layer2'),
    layers.Dense(4, name='layer3')
])

# model.weights # Error

In [8]:
X = tf.ones((1, 4))
y = model(X)

model.weights

[<tf.Variable 'sequential_2/layer1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.38013554, -0.42864633],
        [ 0.9834528 , -0.88952327],
        [-0.7725432 , -0.7129495 ],
        [-0.12196326, -0.13263726]], dtype=float32)>,
 <tf.Variable 'sequential_2/layer1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'sequential_2/layer2/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[ 0.651832  ,  0.58173454,  1.0706286 ],
        [-1.0539346 ,  0.97077775, -0.04235601]], dtype=float32)>,
 <tf.Variable 'sequential_2/layer2/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'sequential_2/layer3/kernel:0' shape=(3, 4) dtype=float32, numpy=
 array([[ 0.38080168,  0.84868777,  0.21695173, -0.07043689],
        [-0.69620883, -0.6199563 ,  0.3662989 , -0.7817523 ],
        [ 0.13444877,  0.34853983,  0.3476026 ,  0.7173135 ]],
       dtype=float32)>,
 <tf.Variable 'sequential_2/layer3/bias:0' shape=(4

- Once a model is 'built', you can call its `summary()` method to display its contents.

In [9]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               multiple                  10        
_________________________________________________________________
layer2 (Dense)               multiple                  9         
_________________________________________________________________
layer3 (Dense)               multiple                  16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


- However, it can be very useful when building a Sequential model incrementally to be able to display the summary of the model so far, including the current output shape. 
- In this case, you should start your model by passing an **`Input` object** to your model, so that it knows its input shape from the start.
    - Note that the `Input` object is not displayed as part of `model.layers`, since it's not a layer.

In [10]:
model = keras.models.Sequential()
model.add(layers.Input(shape=(4, )))
model.add(layers.Dense(2, activation='relu'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.layers

- A simple alternative is to pass an **`input_shape` argument** to the first layer.

In [12]:
model = keras.models.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4,)))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


- Models built with a predefined input shape like this always have weights (even before seeing any data) and always have a defined output shape.
- In general, it's a **recommended best practice** to always **specify the input shape** of a Sequential model in advance if you know what it is.

# Feature extraction with a Sequential model
- Once a Sequential model has been built, it behaves like a Functional API model. 
- This means that every layer has an input and output attribute. 
    - These attributes can be used to do neat things, like quickly creating a model that extracts the outputs of all intermediate layers in a Sequential model.

In [13]:
initial_model = keras.Sequential([
    keras.layers.Input(shape=(250, 250, 3)),
    layers.Conv2D(32, 5, strides=2, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu')
])

In [14]:
feature_extractor = keras.models.Model(
    inputs = initial_model.inputs,
    outputs = [layer.output for layer in initial_model.layers]
)

In [15]:
# Call feature extractor on a test input
X = tf.ones((1, 250, 250, 3))
features = feature_extractor(X)

- Here's a similar example that only extract features from one layer.

In [16]:
initial_model = keras.Sequential([
    keras.layers.Input(shape=(250, 250, 3)),
    layers.Conv2D(32, 5, strides=2, activation='relu'),
    layers.Conv2D(32, 3, activation='relu', name='my_intermediate_layer'),
    layers.Conv2D(32, 3, activation='relu')
])

In [17]:
feature_extractor = keras.models.Model(
    inputs = initial_model.inputs,
    outputs = initial_model.get_layer(name='my_intermediate_layer').output
)

In [18]:
# Call feature extractor on a test input
X = tf.ones((1, 250, 250, 3))
features = feature_extractor(X)

# Transfer learning with a Sequential model
- Transfer learning consists of **freezing the bottom layers** in a model and **only training the top layers**. 
- Here are two common transfer learning blueprint involving Sequential models.
- First, let's say that you have a Sequential model, and you want to freeze all layers except the last one. 
    - In this case, you would simply iterate over `model.layers` and set `layer.trainable = False` on each layer, except the last one.

In [19]:
# model = keras.Sequential([
#     keras.Input(shape=(784)),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(10)
# ])

# Presumably you would want to first load pre-trained weights.
# model.load_weights(...)

# Freeze all layers except the last one.
# for layer in model.layers[:-1]:
#     layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
# model.compile(...)
# model.fit(...)

- Another common blueprint is to use a Sequential model to **stack a pre-trained model and some freshly initialized classification layers**.

In [20]:
# Load a convolutional base model with pre-trained weights
base_model = keras.applications.Xception(
    weights = 'imagenet',
    include_top=False,
    pooling='avg'
)

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.models.Sequential([
    base_model,
    layers.Dense(100)
])

# Compile & train
# model.compile(...)
# model.fit(...)